# **Sales predictions**

In [78]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

In [79]:
train = pd.read_csv(os.getcwd()+"/train.csv")
test = pd.read_csv(os.getcwd()+"/competitionTest.csv")
#oil = pd.read_csv(os.getcwd()+"/data/oil.csv")
#stores = pd.read_csv(os.getcwd()+"/data/stores.csv")
#transactions = pd.read_csv(os.getcwd()+"/data/transactions.csv")
#Holidays = pd.read_csv(os.getcwd()+"/data/holidays_events.csv")


## **train test split**

In [80]:
train["date"] = pd.to_datetime(train["date"])
test["date"] = pd.to_datetime(test["date"])

In [81]:
train = train.loc[train.date.dt.year > 2015]

In [82]:
train.set_index('date', inplace=True)

In [83]:
train.family.replace({k:v for k,v in zip(
    train.family.unique(),
    range(len(train.family.unique()))
)},inplace=True)

In [84]:
test.family.replace({k:v for k,v in zip(
    pd.read_csv(os.getcwd()+"/train.csv").family.unique(),
    range(len(pd.read_csv(os.getcwd()+"/train.csv").family.unique()))
)},inplace=True)

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

preds = pd.DataFrame(columns=["date","store_nbr","family","sales"])


date_array = pd.date_range(start='2017-08-16', end='2017-08-31')


# Iterate over unique combinations of store and family
for (store, family), group_train in train.groupby(['store_nbr', 'family']):

    
    
    # Assuming you have only one time series for each combination (e.g., sales for a specific family and store)
    time_series = group_train['sales']
    

    # Fit an Exponential Smoothing model
    ets_model = ExponentialSmoothing(time_series, trend='add', seasonal='add', seasonal_periods=7)  # Adjust parameters as needed
    ets_result = ets_model.fit()

    # Make predictions
    forecast_horizon = 16  # Adjust based on your forecasting horizon
    forecast_values = ets_result.forecast(steps=forecast_horizon)
    

    ph = pd.DataFrame({"date":date_array,
                      "store_nbr":np.full(16, store),
                      "family":np.full(16, family),
                      "sales":forecast_values.values})
    
    
    preds = pd.concat([preds,ph], ignore_index=True)
    

preds
    

,date,store_nbr,family,sales
0,2017-08-16,1,0,4.306317
1,2017-08-17,1,0,4.398686
2,2017-08-18,1,0,5.711585
3,2017-08-19,1,0,5.001613
4,2017-08-20,1,0,2.649431
...,...,...,...,...
28507,2017-08-27,54,32,2.917842
28508,2017-08-28,54,32,3.317838
28509,2017-08-29,54,32,3.141319
28510,2017-08-30,54,32,3.118925


In [86]:
test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,0,0
1,3000889,2017-08-16,1,1,0
2,3000890,2017-08-16,1,2,2
3,3000891,2017-08-16,1,3,20
4,3000892,2017-08-16,1,4,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,28,1
28508,3029396,2017-08-31,9,29,0
28509,3029397,2017-08-31,9,30,1
28510,3029398,2017-08-31,9,31,9


In [87]:
preds["store_nbr"] = preds.store_nbr.astype(int)
preds["family"] = preds.family.astype(int)

In [88]:
sub = pd.merge(test, preds, on=['date', 'store_nbr', 'family'], how='inner')

In [89]:
sub = sub.loc[:,["id","sales"]]

In [90]:
sub

,id,sales
0,3000888,4.306317e+00
1,3000889,-1.336201e-07
2,3000890,3.947865e+00
3,3000891,2.239365e+03
4,3000892,1.571205e-01
...,...,...
28507,3029395,2.946953e+02
28508,3029396,1.017735e+02
28509,3029397,1.084968e+03
28510,3029398,1.499895e+02


In [91]:
sub.to_csv("submission.csv")